In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
# Read the two CSV files into dataframes
expression = pd.read_csv('../data/raw/OmicsExpressionProteinCodingGenesTPMLogp1.csv')
mutation = pd.read_csv('../data/raw/OmicsSomaticMutations.csv')

C:\Users\hp\AppData\Local\Temp\ipykernel_1332\1829342583.py:3: DtypeWarning: Columns (22,50,56,57,58,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  mutation = pd.read_csv('../data/raw/OmicsSomaticMutations.csv')


In [3]:
mutation = mutation[mutation['HugoSymbol'] == 'TP53']  # filter only TP53 mutations
mutation_subset = mutation[['ModelID', 'VariantType']]  # keep only needed columns

expression = expression.rename(columns={expression.columns[0]: 'ModelID'})
expression.drop_duplicates(inplace=True)

# --- Merge datasets ---
merged_df = pd.merge(expression, mutation_subset, on='ModelID', how='left')

# --- Encode mutation type (multiclass target) ---
le = LabelEncoder()
merged_df['VariantType'] = le.fit_transform(merged_df['VariantType'].astype(str))  # Encode NaN as a class if needed

# Save class label mapping
class_names = {index: label for index, label in enumerate(le.classes_)}
print("Class labels:", class_names)

# --- Binary label: is TP53 mutated or not ---
merged_df['Mutated'] = np.where(merged_df['VariantType'] == le.transform(['nan'])[0], 0, 1)  # 0 if VariantType was NaN, else 1
merged_df.columns = merged_df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Drop text in parentheses and strip whitespace


Class labels: {0: 'SNV', 1: 'deletion', 2: 'insertion', 3: 'nan', 4: 'substitution'}


In [4]:
merged_df.to_csv("../data/processed/merged_data.csv")

In [5]:
merged_df

,ModelID,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1,VariantType,Mutated
0,ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,...,0.201634,2.130931,0.555816,0.000000,0.275007,0.0,0.000000,0.000000,0,1
1,ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,...,0.321928,1.464668,0.632268,0.000000,0.014355,0.0,0.000000,0.000000,3,0
2,ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,...,0.831877,2.946731,0.475085,0.000000,0.084064,0.0,0.000000,0.042644,0,1
3,ACH-001979,4.240314,0.000000,5.681168,2.063503,1.641546,0.000000,7.236301,5.948367,4.596935,...,0.847997,1.887525,0.176323,0.000000,0.000000,0.0,0.000000,0.000000,3,0
4,ACH-002438,4.032101,0.000000,6.674687,2.117695,3.003602,0.000000,2.169925,6.838195,3.253989,...,0.584963,1.748461,0.839960,0.000000,0.014355,0.0,0.000000,0.042644,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790,ACH-000285,0.056584,0.000000,6.604368,3.266037,4.973152,0.411426,0.097611,0.704872,4.829850,...,1.310340,3.039138,0.344828,0.000000,0.000000,0.0,0.475085,0.042644,1,1
1791,ACH-002669,3.111031,0.000000,7.031329,1.541019,3.664483,0.014355,3.624101,6.805421,4.472488,...,0.356144,1.327687,0.000000,0.000000,0.014355,0.0,0.000000,0.000000,0,1
1792,ACH-001858,4.390943,0.000000,7.013239,1.887525,3.252476,0.028569,3.286881,6.902194,5.410748,...,0.613532,1.992768,0.704872,0.000000,1.464668,0.0,0.000000,0.526069,0,1
1793,ACH-001997,5.057450,0.000000,7.815191,2.538538,3.893362,0.028569,4.079805,6.971659,4.469886,...,1.292782,2.153805,0.687061,0.000000,0.000000,0.0,0.000000,0.000000,0,1
